## Day 10 - Pipes!

Pipes are one large continuous loop.

E.g. Square loop of pipe:
```
.....
.F-7.
.|.|.
.L-J.
.....
```

Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?

In [ ]:
with open("./example-4-steps.txt") as f:
    example_4_lines = [line.strip() for line in f.readlines()]
with open("./example-8-steps.txt") as f:
    example_8_lines = [line.strip() for line in f.readlines()]
with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]


example_4_lines

Idea:

 - 2D grid. Find that vector class from snake game.
 - Treat bends/pipes like instructions (i.e. + vector (1 0) for a straight up or something along those lines)

In [ ]:
class Vector2D():
    def __init__(self, y, x):
        """
        np grid works like 
        (y,x)    (y, x+1)    (y, x+2)
        (y+1, x) (y+1, x+1)  (y+1, x+2)
        (y+2, x) .......
        """
        self.y = y
        self.x = x

    def __add__(self, v2):
        return Vector2D(self.x + v2.x, self.y + v2.y)
    
    def __sub__(self, v2):
        return Vector2D(self.x - v2.x, self.y - v2.y)

    def __iadd__(self, v2):
        self.x += v2.x
        self.y += v2.y
        return self
    
    def __isub__(self, v2):
        self.x -= v2.x
        self.y -= v2.y
        return self
    
    def __str__(self):
        return f'({self.x},{self.y})'
    
    def __eq__(self, v2):
        return (self.x==v2.x and self.y==v2.y)

    @property
    def as_tuple(self):
        return (self.x, self.y)

In [ ]:
def create_grid_map(lines: list[str]) -> tuple[dict, tuple[int, int]]:
    """
    Takes in the input text (separated by line into a list)
    and notes the value at each location in a dict

    so key in like position (1,2) and get value at that location.

    USING SYSTEM:
        (y, x)
        where (0,0) is top left corner

    Returns dict map and also the starting position!
    """
    grid_map = {}
    starting_position = None
    for y_index, line in enumerate(lines):
        for x_index, val in enumerate(line):
            if val == "S":
                starting_position = (y_index, x_index)
            grid_map[(y_index, x_index)] = val

    if starting_position is None:
        raise ValueError("Didn't find starting position!")
    return grid_map, starting_position

In [ ]:
import copy

class Plumber():
    """
    Who else would be going through the pipes?!
    """
    UP = Vector2D(-1,0)
    DOWN = Vector2D(1,0)
    LEFT = Vector2D(0,-1)
    RIGHT = Vector2D(0,1)

    def __init__(self, starting_position: Vector2D, current_dir: Vector2D):
        starting_position = copy.deepcopy(starting_position)
        self.pos: Vector2D = starting_position
        self.current_dir: Vector2D = current_dir
    
    def move(self, dir : Vector2D):
        """
        moves the Plumber in chosen direction.
        """

        # Dont go backwards surely
        if self.current_dir == self.RIGHT and dir == self.LEFT:
            raise ValueError("Backwards!!!")
        if self.current_dir == self.LEFT and dir == self.RIGHT:
            raise ValueError("Backwards!!!")
        if self.current_dir == self.UP and dir == self.DOWN:
            raise ValueError("Backwards!!!")
        if self.current_dir == self.DOWN and dir == self.UP:
            raise ValueError("Backwards!!!")

        # Set new current direction
        self.current_dir = dir
        # Update position
        self.pos += self.current_dir

- `|` is a vertical pipe connecting north and south.
- `-` is a horizontal pipe connecting east and west.
- `L` is a 90-degree bend connecting north and east.
- `J` is a 90-degree bend connecting north and west.
- `7` is a 90-degree bend connecting south and west.
- `F` is a 90-degree bend connecting south and east.
- `.` is ground; there is no pipe in this tile.
- `S` is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.

In [ ]:
def get_new_direction_from_instruction_and_relative_pos(instruction: str, relative_position: Vector2D) -> Vector2D:
    """
    relative_position means instruction_pos - current_pos
    """
    if instruction == "|":
        if relative_position == Plumber.UP:
            return relative_position
        elif relative_position == Plumber.DOWN:
            return relative_position
        else:
            raise ValueError("|")
    if instruction == "-":
        if relative_position == Plumber.LEFT:
            return relative_position
        elif relative_position == Plumber.RIGHT:
            return relative_position
        else:
            raise ValueError("-")
    if instruction == "L":
        if relative_position == Plumber.DOWN:
            return Plumber.RIGHT
        elif relative_position == Plumber.LEFT:
            return Plumber.UP
        else:
            raise ValueError("L")
    if instruction == "J":
        if relative_position == Plumber.DOWN:
            return Plumber.LEFT
        elif relative_position == Plumber.RIGHT:
            return Plumber.UP
        else:
            raise ValueError("J")
    if instruction == "7":
        if relative_position == Plumber.UP:
            return Plumber.LEFT
        elif relative_position == Plumber.RIGHT:
            return Plumber.DOWN
    if instruction == "F":
        if relative_position == Plumber.UP:
            return Plumber.RIGHT
        elif relative_position == Plumber.LEFT:
            return Plumber.DOWN
    
    raise ValueError(f"What are we doing with {instruction} here?")


In [ ]:
def find_first_moves(grid_map: dict, starting_position: Vector2D) -> tuple[Vector2D, Vector2D]:
    assert grid_map[starting_position.as_tuple] == "S"

    above_val = grid_map[(starting_position + Plumber.UP).as_tuple]
    below_val = grid_map[(starting_position + Plumber.DOWN).as_tuple]
    left_val = grid_map[(starting_position + Plumber.LEFT).as_tuple]
    right_val = grid_map[(starting_position + Plumber.RIGHT).as_tuple]

    print("A",above_val, "B",below_val, "L",left_val, "R",right_val)

    initial_direcitons = []
    try:
        direction = get_new_direction_from_instruction_and_relative_pos(above_val, Plumber.UP)
        initial_direcitons.append(direction)
    except:
        pass
    try:
        direction = get_new_direction_from_instruction_and_relative_pos(below_val, Plumber.DOWN)
        initial_direcitons.append(direction)
    except:
        pass
    try:
        direction = get_new_direction_from_instruction_and_relative_pos(left_val, Plumber.LEFT)
        initial_direcitons.append(direction)
    except:
        pass
    try:
        direction = get_new_direction_from_instruction_and_relative_pos(right_val, Plumber.RIGHT)
        initial_direcitons.append(direction)
    except:
        pass

    assert len(initial_direcitons) == 2, initial_direcitons

    return tuple(initial_direcitons)

In [ ]:
def part1(lines: list[str]) -> int:
    grid_map, starting_position_tuple = create_grid_map(lines)
    starting_position = Vector2D(y=starting_position_tuple[0], x=starting_position_tuple[1])

    move1, move2 = find_first_moves(grid_map, starting_position)
    plumber1 = Plumber(starting_position, move1)
    plumber2 = Plumber(starting_position, move2)
    plumber1.move(move1)
    plumber2.move(move2)
    print(plumber1.pos, grid_map[plumber1.pos.as_tuple])
    print(plumber2.pos, grid_map[plumber2.pos.as_tuple])



    

part1(example_4_lines)

need to fix relative position etc. etc.